In [1]:
import pandas as pd
import numpy as np
import os

from sklearn.ensemble import IsolationForest

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from sklearn.metrics import *
from collections import Counter

In [2]:
class iForest:

    def __init__(self):
        self.ACs = []
        for ac_name in os.listdir('./ACs'):
            ac_i = pd.read_pickle('./ACs/'+ac_name)
            self.ACs.append(ac_i)
        self.n_rows = len(self.ACs[0])

    def get_X_Y(self, row: int) -> (pd.DataFrame, pd.DataFrame):
        data = []
        for AC in self.ACs:
            AC = pd.DataFrame(AC)
            data.append(AC.iloc[row,:].tolist())
        data = pd.DataFrame(data).reset_index(drop=True)
        X = data.iloc[:,1:]
        Y = data.iloc[:,0]
        
        return X, Y.astype(int).tolist()   
    
    def cluster(self, x_row: pd.DataFrame) -> (list):
                
        clf = IsolationForest().fit(x_row)
        y_hat = clf.decision_function(x_row)
        labels = 1 - (y_hat - min(y_hat))/(max(y_hat) - min(y_hat))
                    
        anomaly_score = 1 - (y_hat - min(y_hat))/(max(y_hat) - min(y_hat))
        
        return anomaly_score.tolist()
    
    def cluster_all(self, do_score=False) -> (list, list):
        
        anomaly_score, y = [], []
        
        for row in range(self.n_rows):
            print(row)
            x_row, y_row = self.get_X_Y(row)
            if sum(y_row) >= len(y_row)//2: 
                print('oops')

            anomaly_score_row = self.cluster(x_row)
            anomaly_score += anomaly_score_row
            y += y_row
                    
        return anomaly_score, y
    
    def auc_score(self, anomaly_score: list, y: list):
        # AUC
        fpr, tpr, thresholds = roc_curve(y, anomaly_score)
        AUC = auc(fpr, tpr)
        print('AUC score: %f' % AUC)
    
    def score(self, anomaly_score: list, y: list, threashold=0.8) -> k:
        
        yhat = [ 1 if y_i >= threashold else 0 for y_i in anomaly_score]
        
        # accuracy: (tp + tn) / (p + n)
        accuracy = accuracy_score(y, yhat)
        print('Accuracy: %f' % accuracy)
        # precision tp / (tp + fp)
        precision = precision_score(y, yhat)
        print('Precision: %f' % precision)
        # recall: tp / (tp + fn) More important in this case
        recall = recall_score(y, yhat)
        print('Recall: %f' % recall)
        # f1: 2 tp / (2 tp + fp + fn)
        f1 = f1_score(y, yhat)
        print('F1 score: %f' % f1)
        return f1

NameError: name 'k' is not defined

In [ ]:
ifo = iForest()
anomaly_score, y = ifo.cluster_all()
ifo.auc_score(anomaly_score, y)
for p in [0.5,0.6,0.7,0.8,0.9]:
    print(p)
    ifo.score(anomaly_score, y, threashold=p)